# Import Packages

In [63]:
import nltk
from nameparser.parser import HumanName


In [64]:
import logging
import os
import glob2
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import nltk
import pandas as pd
import re
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')',
                   '[', ']', '{', '}','%','&','+','$','@','~','-','_','://','--','[]','`']) # remove it if you need punctuation 
from nltk.tokenize import RegexpTokenizer
from datetime import datetime

# Library setup

In [65]:
cd E:\task

E:\task


In [66]:
listdir = []
listdir = glob2.glob("*.txt")

In [67]:
listdir

['wifi_data.txt']

In [68]:
data = pd.read_csv(listdir[0], sep="`", header=None)

In [69]:
data.columns = ["time", "range", "bssid", "type", "name", "etc1", "etc2"]

In [70]:
data

,time,range,bssid,type,name,etc1,etc2
0,2017-11-08 10:02,-27dB,84:3a:4b:04:d6:66,PREQ,NaN,NaN,NaN
1,2017-11-08 10:02,-40dB,88:79:7e:eb:1c:1f,PREQ,NaN,NaN,NaN
2,2017-11-08 10:02,-54dB,b8:27:eb:28:39:45,PREQ,NaN,NaN,NaN
3,2017-11-08 10:02,-57dB,74:da:38:92:5f:b4,PREQ,NaN,NaN,NaN
4,2017-11-08 10:02,-58dB,88:79:7e:74:5b:de,PREQ,NaN,NaN,NaN
5,2017-11-08 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,TP-LINK_3166,NaN,NaN
6,2017-11-08 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,NaN,NaN,NaN
7,2017-11-08 10:02,-60dB,e4:02:9b:22:db:5c,PREQ,NaN,NaN,NaN
8,2017-11-08 10:02,-60dB,a4:70:d6:7a:ac:c5,PREQ,NaN,NaN,NaN
9,2017-11-08 10:02,-62dB,38:71:de:de:9f:ac,PREQ,NaN,NaN,NaN


In [71]:
data.to_csv("input_data.csv", index=False)

In [73]:
inp = pd.read_csv( "input_data_with_lat_lon.csv" )

In [74]:
inp

,time,range,bssid,type,name,etc1,etc2,lat,lon
0,11/8/2017 10:02,-27dB,84:3a:4b:04:d6:66,PREQ,NaN,NaN,NaN,75.46295,150.47512
1,11/8/2017 10:02,-40dB,88:79:7e:eb:1c:1f,PREQ,NaN,NaN,NaN,55.30440,12.95574
2,11/8/2017 10:02,-54dB,b8:27:eb:28:39:45,PREQ,NaN,NaN,NaN,53.17357,-172.24221
3,11/8/2017 10:02,-57dB,74:da:38:92:5f:b4,PREQ,NaN,NaN,NaN,-50.42641,-164.46908
4,11/8/2017 10:02,-58dB,88:79:7e:74:5b:de,PREQ,NaN,NaN,NaN,-61.78642,-179.85250
5,11/8/2017 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,TP-LINK_3166,NaN,NaN,-74.42309,-17.17440
6,11/8/2017 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,NaN,NaN,NaN,60.83873,38.52055
7,11/8/2017 10:02,-60dB,e4:02:9b:22:db:5c,PREQ,NaN,NaN,NaN,-20.20166,-51.22016
8,11/8/2017 10:02,-60dB,a4:70:d6:7a:ac:c5,PREQ,NaN,NaN,NaN,27.26680,152.82620
9,11/8/2017 10:02,-62dB,38:71:de:de:9f:ac,PREQ,NaN,NaN,NaN,-23.89006,135.27577


In [75]:
inp['lat']

0     75.46295
1     55.30440
2     53.17357
3    -50.42641
4    -61.78642
5    -74.42309
6     60.83873
7    -20.20166
8     27.26680
9    -23.89006
10    42.75275
11    35.90031
12   -47.74699
13    48.60508
14   -56.84606
15    79.54283
16   -67.14141
17    80.93490
18   -35.16129
19    87.08712
20   -65.30015
21    67.01283
22    16.11715
23    85.18808
24    -3.11911
25   -37.72191
26    62.79562
27   -58.03426
28    21.21035
29    89.44544
Name: lat, dtype: float64

In [76]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)

Backwerk, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


# Now try to get some features via lat lon

In [80]:
import geocoder
for i in range(len(inp)):
    l = inp['lat'][i]
    k = inp['lon'][i]
    location = geocoder.google([l , k], method='reverse')
    inp['city'][i] = location.city
    inp['state'][i] = location.state
    inp['state_long'][i] = location.state_long
    inp['country'][i] = location.country
    inp['country_long'][i] = location.country_long

C:\Users\ashish\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ashish\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ashish\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ashish\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [79]:
inp['city'] = location.city
inp['state'] = location.state
inp['state_long'] = location.state_long
inp['country'] = location.country
inp['country_long'] = location.country_long

In [81]:
data = inp
data

,time,range,bssid,type,name,etc1,etc2,lat,lon,city,state,state_long,country,country_long
0,11/8/2017 10:02,-27dB,84:3a:4b:04:d6:66,PREQ,NaN,NaN,NaN,75.46295,150.47512,None,None,None,RU,Russia
1,11/8/2017 10:02,-40dB,88:79:7e:eb:1c:1f,PREQ,NaN,NaN,NaN,55.30440,12.95574,Höllviken,Skåne län,Skåne län,SE,Sweden
2,11/8/2017 10:02,-54dB,b8:27:eb:28:39:45,PREQ,NaN,NaN,NaN,53.17357,-172.24221,None,None,None,US,United States
3,11/8/2017 10:02,-57dB,74:da:38:92:5f:b4,PREQ,NaN,NaN,NaN,-50.42641,-164.46908,None,None,None,None,None
4,11/8/2017 10:02,-58dB,88:79:7e:74:5b:de,PREQ,NaN,NaN,NaN,-61.78642,-179.85250,None,None,None,None,None
5,11/8/2017 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,TP-LINK_3166,NaN,NaN,-74.42309,-17.17440,None,None,None,None,None
6,11/8/2017 10:02,-58dB,ec:01:ee:cc:74:ec,PREQ,NaN,NaN,NaN,60.83873,38.52055,None,Arkhangelskaya oblast',Arkhangelskaya oblast',RU,Russia
7,11/8/2017 10:02,-60dB,e4:02:9b:22:db:5c,PREQ,NaN,NaN,NaN,-20.20166,-51.22016,None,MS,Mato Grosso do Sul,BR,Brazil
8,11/8/2017 10:02,-60dB,a4:70:d6:7a:ac:c5,PREQ,NaN,NaN,NaN,27.26680,152.82620,None,None,None,None,None
9,11/8/2017 10:02,-62dB,38:71:de:de:9f:ac,PREQ,NaN,NaN,NaN,-23.89006,135.27577,None,None,None,None,None


In [82]:
data['time'] = pd.to_datetime(data['time'])

In [83]:
data['new_date'] = [d.date() for d in data['time']]
data['new_time'] = [d.time() for d in data['time']]

In [84]:
data

,time,range,bssid,type,name,etc1,etc2,lat,lon,city,state,state_long,country,country_long,new_date,new_time
0,2017-11-08 10:02:00,-27dB,84:3a:4b:04:d6:66,PREQ,NaN,NaN,NaN,75.46295,150.47512,None,None,None,RU,Russia,2017-11-08,10:02:00
1,2017-11-08 10:02:00,-40dB,88:79:7e:eb:1c:1f,PREQ,NaN,NaN,NaN,55.30440,12.95574,Höllviken,Skåne län,Skåne län,SE,Sweden,2017-11-08,10:02:00
2,2017-11-08 10:02:00,-54dB,b8:27:eb:28:39:45,PREQ,NaN,NaN,NaN,53.17357,-172.24221,None,None,None,US,United States,2017-11-08,10:02:00
3,2017-11-08 10:02:00,-57dB,74:da:38:92:5f:b4,PREQ,NaN,NaN,NaN,-50.42641,-164.46908,None,None,None,None,None,2017-11-08,10:02:00
4,2017-11-08 10:02:00,-58dB,88:79:7e:74:5b:de,PREQ,NaN,NaN,NaN,-61.78642,-179.85250,None,None,None,None,None,2017-11-08,10:02:00
5,2017-11-08 10:02:00,-58dB,ec:01:ee:cc:74:ec,PREQ,TP-LINK_3166,NaN,NaN,-74.42309,-17.17440,None,None,None,None,None,2017-11-08,10:02:00
6,2017-11-08 10:02:00,-58dB,ec:01:ee:cc:74:ec,PREQ,NaN,NaN,NaN,60.83873,38.52055,None,Arkhangelskaya oblast',Arkhangelskaya oblast',RU,Russia,2017-11-08,10:02:00
7,2017-11-08 10:02:00,-60dB,e4:02:9b:22:db:5c,PREQ,NaN,NaN,NaN,-20.20166,-51.22016,None,MS,Mato Grosso do Sul,BR,Brazil,2017-11-08,10:02:00
8,2017-11-08 10:02:00,-60dB,a4:70:d6:7a:ac:c5,PREQ,NaN,NaN,NaN,27.26680,152.82620,None,None,None,None,None,2017-11-08,10:02:00
9,2017-11-08 10:02:00,-62dB,38:71:de:de:9f:ac,PREQ,NaN,NaN,NaN,-23.89006,135.27577,None,None,None,None,None,2017-11-08,10:02:00


In [86]:
import sys
import operator
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection, metrics, ensemble
import xgboost as xgb


# Features

In [87]:
df = data
df['Year'] = df['time'].dt.year
df['Month'] = df['time'].dt.month
df['Day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute

In [88]:
df

,time,range,bssid,type,name,etc1,etc2,lat,lon,city,...,state_long,country,country_long,new_date,new_time,Year,Month,Day,hour,minute
0,2017-11-08 10:02:00,-27dB,84:3a:4b:04:d6:66,PREQ,NaN,NaN,NaN,75.46295,150.47512,None,...,None,RU,Russia,2017-11-08,10:02:00,2017,11,8,10,2
1,2017-11-08 10:02:00,-40dB,88:79:7e:eb:1c:1f,PREQ,NaN,NaN,NaN,55.30440,12.95574,Höllviken,...,Skåne län,SE,Sweden,2017-11-08,10:02:00,2017,11,8,10,2
2,2017-11-08 10:02:00,-54dB,b8:27:eb:28:39:45,PREQ,NaN,NaN,NaN,53.17357,-172.24221,None,...,None,US,United States,2017-11-08,10:02:00,2017,11,8,10,2
3,2017-11-08 10:02:00,-57dB,74:da:38:92:5f:b4,PREQ,NaN,NaN,NaN,-50.42641,-164.46908,None,...,None,None,None,2017-11-08,10:02:00,2017,11,8,10,2
4,2017-11-08 10:02:00,-58dB,88:79:7e:74:5b:de,PREQ,NaN,NaN,NaN,-61.78642,-179.85250,None,...,None,None,None,2017-11-08,10:02:00,2017,11,8,10,2
5,2017-11-08 10:02:00,-58dB,ec:01:ee:cc:74:ec,PREQ,TP-LINK_3166,NaN,NaN,-74.42309,-17.17440,None,...,None,None,None,2017-11-08,10:02:00,2017,11,8,10,2
6,2017-11-08 10:02:00,-58dB,ec:01:ee:cc:74:ec,PREQ,NaN,NaN,NaN,60.83873,38.52055,None,...,Arkhangelskaya oblast',RU,Russia,2017-11-08,10:02:00,2017,11,8,10,2
7,2017-11-08 10:02:00,-60dB,e4:02:9b:22:db:5c,PREQ,NaN,NaN,NaN,-20.20166,-51.22016,None,...,Mato Grosso do Sul,BR,Brazil,2017-11-08,10:02:00,2017,11,8,10,2
8,2017-11-08 10:02:00,-60dB,a4:70:d6:7a:ac:c5,PREQ,NaN,NaN,NaN,27.26680,152.82620,None,...,None,None,None,2017-11-08,10:02:00,2017,11,8,10,2
9,2017-11-08 10:02:00,-62dB,38:71:de:de:9f:ac,PREQ,NaN,NaN,NaN,-23.89006,135.27577,None,...,None,None,None,2017-11-08,10:02:00,2017,11,8,10,2


In [90]:
df.to_csv("output_featured.csv")